In [24]:
import sys
sys.path.append('../RecSysRep/')

In [25]:
import Basics.Load as ld

URM_all, _, _, _, _ = ld.getCOOs()
ICM_all = ld.getICMall()

SEED = 6180

In [26]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.7, seed=SEED)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13649 ( 0.0%) Users that have less than 1 test interactions


In [27]:
import os

ofp = "../models_temp/Similarity_Hybrid/Split07/"


models_to_combine_best = {
                            'IALS' : {"num_factors": 29, "epochs": 50, "confidence_scaling": "log", "alpha": 0.001, "epsilon": 0.001, "reg": 0.01},
                            'SLIMBPR' : {"epochs": 440, "lambda_i": 0.007773815998802306, "lambda_j": 0.003342522366982381, "learning_rate": 0.010055161410725193, "topK": 4289, "random_seed": 1234, "sgd_mode": "sgd"},
                            'SLIMER': {'topK': 6000, 'l1_ratio': 0.0005495104968035837, 'alpha': 0.08007142704041009, 'workers': 8},
                            'P3alpha': {'topK': 4834, 'alpha': 1.764994849187595, 'normalize_similarity': True, 'implicit': True},
                            'RP3beta': {"topK": 1049, "alpha": 1.1626473723475605, "beta": 0.6765017195261293, "normalize_similarity": True, "implicit": True},
                            'ICM_all': {"shrink": 5675, "topK": 2310, "feature_weighting": "BM25", "normalize": False},
                            'TopPop': {},
                            'PureSVD': {'num_factors': 30},
                            'UserKNN': {"topK": 469, "similarity": "cosine", "shrink": 588, "normalize": True, "feature_weighting": "TF-IDF", "URM_bias": False},
                            'EASE_R': {'l2_norm': 3750},
}

In [28]:
def model_init(recommender, name, args):
    path_name = ofp + name + str(SEED) + ".zip"
    print(path_name)
    if os.path.exists(path_name):
        print("Model found!")
        recommender.load_model(ofp, name + str(SEED))
    else:
        print("Model does not exists, creating...")
        if not os.path.exists(ofp):
            print("Main folder does not exist, creating...")
            os.makedirs(ofp)
        recommender.fit(**args)
        recommender.save_model(ofp, name + str(SEED))

In [29]:
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

In [30]:
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridTwoRecommender
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridMultipleRecommender

In [31]:
recommender1 = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
recommender2 = SLIM_BPR_Cython(URM_train)
recommender3 = RP3betaRecommender(URM_train)
recommender4 = ItemKNNCBFRecommender(URM_train, ICM_all)
recommender5 = IALSRecommender(URM_train)
recommender6 = P3alphaRecommender(URM_train)
recommender7 = UserKNNCFRecommender(URM_train)
recommender8 = PureSVDRecommender(URM_train)
recommender9 = TopPop(URM_train)
recommender10 = EASE_R_Recommender(URM_train)

model_init(recommender1, 'SLIMER', models_to_combine_best['SLIMER'])
model_init(recommender2, 'SLIMBPR', models_to_combine_best['SLIMBPR'])
model_init(recommender3, 'RP3beta', models_to_combine_best['RP3beta'])
model_init(recommender4, 'ICM_all', models_to_combine_best['ICM_all'])
model_init(recommender5, 'IALS', models_to_combine_best['IALS'])
model_init(recommender6, 'P3alpha', models_to_combine_best['P3alpha'])
model_init(recommender7, 'UserKNN', models_to_combine_best['UserKNN'])
model_init(recommender8, 'PureSVD', models_to_combine_best['PureSVD'])
model_init(recommender9, 'TopPop', models_to_combine_best['TopPop'])
model_init(recommender10, 'EASE_R', models_to_combine_best['EASE_R'])

  0%|                                                                                        | 0/18059 [00:00<?, ?it/s]

../models_temp/Similarity_Hybrid/Split07/SLIMER6180.zip
Model does not exists, creating...
Main folder does not exist, creating...


100%|███████████████████████████████████████████████████████████████████████████▊| 18000/18059 [13:43<00:02, 28.59it/s]

SLIMElasticNetRecommender: Saving model in file '../models_temp/Similarity_Hybrid/Split07/SLIMER6180'


100%|███████████████████████████████████████████████████████████████████████████▉| 18056/18059 [14:00<00:00, 28.59it/s]

...

Processed 13650 (100.0%) in 0.92 sec. BPR loss is 2.77E+01. Sample per second: 14783
SLIM_BPR_Recommender: Epoch 222 of 440. Elapsed time 1.42 min
Processed 13650 (100.0%) in 1.31 sec. BPR loss is 2.75E+01. Sample per second: 10408
SLIM_BPR_Recommender: Epoch 223 of 440. Elapsed time 1.42 min
Processed 13650 (100.0%) in 0.69 sec. BPR loss is 2.74E+01. Sample per second: 19739
SLIM_BPR_Recommender: Epoch 224 of 440. Elapsed time 1.43 min
Processed 13650 (100.0%) in 1.08 sec. BPR loss is 2.74E+01. Sample per second: 12667
SLIM_BPR_Recommender: Epoch 225 of 440. Elapsed time 1.43 min
Processed 13650 (100.0%) in 0.46 sec. BPR loss is 2.72E+01. Sample per second: 29565
SLIM_BPR_Recommender: Epoch 226 of 440. Elapsed time 1.44 min
Processed 13650 (100.0%) in 0.84 sec. BPR loss is 2.66E+01. Sample per second: 16158
SLIM_BPR_Recommender: Epoch 227 of 440. Elapsed time 1.45 min
Processed 13650 (100.0%) in 1.23 sec. BPR loss is 2.72E+01. Sample per second: 11107
SLIM_BPR_Recommender: Epoch 228 o

Processed 13650 (100.0%) in 1.31 sec. BPR loss is 2.77E+01. Sample per second: 10379
SLIM_BPR_Recommender: Epoch 278 of 440. Elapsed time 1.77 min
Processed 13650 (100.0%) in 0.70 sec. BPR loss is 2.87E+01. Sample per second: 19549
SLIM_BPR_Recommender: Epoch 279 of 440. Elapsed time 1.78 min
Processed 13650 (100.0%) in 1.08 sec. BPR loss is 2.83E+01. Sample per second: 12670
SLIM_BPR_Recommender: Epoch 280 of 440. Elapsed time 1.78 min
Processed 13650 (100.0%) in 0.46 sec. BPR loss is 2.78E+01. Sample per second: 29775
SLIM_BPR_Recommender: Epoch 281 of 440. Elapsed time 1.79 min
Processed 13650 (100.0%) in 0.84 sec. BPR loss is 2.80E+01. Sample per second: 16279
SLIM_BPR_Recommender: Epoch 282 of 440. Elapsed time 1.80 min
Processed 13650 (100.0%) in 1.22 sec. BPR loss is 2.77E+01. Sample per second: 11156
SLIM_BPR_Recommender: Epoch 283 of 440. Elapsed time 1.80 min
Processed 13650 (100.0%) in 0.60 sec. BPR loss is 2.85E+01. Sample per second: 22611
SLIM_BPR_Recommender: Epoch 284 o

Processed 13650 (100.0%) in 0.99 sec. BPR loss is 2.93E+01. Sample per second: 13761
SLIM_BPR_Recommender: Epoch 334 of 440. Elapsed time 2.13 min
Processed 13650 (100.0%) in 1.37 sec. BPR loss is 2.91E+01. Sample per second: 9949
SLIM_BPR_Recommender: Epoch 335 of 440. Elapsed time 2.14 min
Processed 13650 (100.0%) in 0.75 sec. BPR loss is 2.90E+01. Sample per second: 18125
SLIM_BPR_Recommender: Epoch 336 of 440. Elapsed time 2.15 min
Processed 13650 (100.0%) in 1.14 sec. BPR loss is 2.85E+01. Sample per second: 12003
SLIM_BPR_Recommender: Epoch 337 of 440. Elapsed time 2.15 min
Processed 13650 (100.0%) in 0.52 sec. BPR loss is 2.91E+01. Sample per second: 26185
SLIM_BPR_Recommender: Epoch 338 of 440. Elapsed time 2.16 min
Processed 13650 (100.0%) in 0.91 sec. BPR loss is 2.89E+01. Sample per second: 15076
SLIM_BPR_Recommender: Epoch 339 of 440. Elapsed time 2.17 min
Processed 13650 (100.0%) in 1.29 sec. BPR loss is 2.86E+01. Sample per second: 10602
SLIM_BPR_Recommender: Epoch 340 of

Processed 13650 (100.0%) in 0.42 sec. BPR loss is 2.93E+01. Sample per second: 32422
SLIM_BPR_Recommender: Epoch 390 of 440. Elapsed time 2.49 min
Processed 13650 (100.0%) in 0.80 sec. BPR loss is 2.96E+01. Sample per second: 16997
SLIM_BPR_Recommender: Epoch 391 of 440. Elapsed time 2.50 min
Processed 13650 (100.0%) in 1.19 sec. BPR loss is 2.95E+01. Sample per second: 11498
SLIM_BPR_Recommender: Epoch 392 of 440. Elapsed time 2.50 min
Processed 13650 (100.0%) in 0.57 sec. BPR loss is 2.97E+01. Sample per second: 24020
SLIM_BPR_Recommender: Epoch 393 of 440. Elapsed time 2.51 min
Processed 13650 (100.0%) in 0.95 sec. BPR loss is 2.89E+01. Sample per second: 14378
SLIM_BPR_Recommender: Epoch 394 of 440. Elapsed time 2.52 min
Processed 13650 (100.0%) in 1.34 sec. BPR loss is 2.94E+01. Sample per second: 10214
SLIM_BPR_Recommender: Epoch 395 of 440. Elapsed time 2.52 min
Processed 13650 (100.0%) in 0.72 sec. BPR loss is 2.99E+01. Sample per second: 18945
SLIM_BPR_Recommender: Epoch 396 o

ItemKNNCBFRecommender: Saving complete
../models_temp/Similarity_Hybrid/Split07/IALS6180.zip
Model does not exists, creating...
IALSRecommender: Epoch 1 of 50. Elapsed time 3.44 sec
IALSRecommender: Epoch 2 of 50. Elapsed time 6.87 sec
IALSRecommender: Epoch 3 of 50. Elapsed time 10.25 sec
IALSRecommender: Epoch 4 of 50. Elapsed time 13.68 sec
IALSRecommender: Epoch 5 of 50. Elapsed time 17.00 sec
IALSRecommender: Epoch 6 of 50. Elapsed time 20.39 sec
IALSRecommender: Epoch 7 of 50. Elapsed time 23.78 sec
IALSRecommender: Epoch 8 of 50. Elapsed time 27.23 sec
IALSRecommender: Epoch 9 of 50. Elapsed time 30.63 sec
IALSRecommender: Epoch 10 of 50. Elapsed time 34.01 sec
IALSRecommender: Epoch 11 of 50. Elapsed time 37.46 sec
IALSRecommender: Epoch 12 of 50. Elapsed time 40.84 sec
IALSRecommender: Epoch 13 of 50. Elapsed time 44.25 sec
IALSRecommender: Epoch 14 of 50. Elapsed time 47.63 sec
IALSRecommender: Epoch 15 of 50. Elapsed time 50.98 sec
IALSRecommender: Epoch 16 of 50. Elapsed ti

In [36]:


recommenderHybrid1 = ItemKNNScoresHybridTwoRecommender(URM_train, recommender4, recommender9)
recommenderHybrid1.fit(alpha = 0.65)

recommenderHybrid2 = ItemKNNScoresHybridTwoRecommender(URM_train, recommenderHybrid1, recommender3)
recommenderHybrid2.fit(alpha = 0.10)

recommenderHybrid3 = ItemKNNScoresHybridTwoRecommender(URM_train, recommenderHybrid2, recommender2)
recommenderHybrid3.fit(alpha = 0.20)

recommenderHybrid4 = ItemKNNScoresHybridTwoRecommender(URM_train, recommenderHybrid3, recommender7)
recommenderHybrid4.fit(alpha = 0.25)

recommenderHybrid5 = ItemKNNScoresHybridTwoRecommender(URM_train, recommenderHybrid4, recommender5)
recommenderHybrid5.fit(alpha = 0.9)

recommenderHybrid6 = ItemKNNScoresHybridTwoRecommender(URM_train, recommenderHybrid5, recommender10)
recommenderHybrid6.fit(alpha = 0.45)

recommenderHybrid7 = ItemKNNScoresHybridTwoRecommender(URM_train, recommenderHybrid6, recommender1)
recommenderHybrid7.fit(alpha = 0.5)

In [34]:
# 0.344935
recommenderHybrid3 = ItemKNNScoresHybridTwoRecommender(URM_train, recommender3, recommender2)
recommenderHybrid3.fit(alpha = 0.20)

recommenderHybrid4 = ItemKNNScoresHybridTwoRecommender(URM_train, recommenderHybrid3, recommender7)
recommenderHybrid4.fit(alpha = 0.25)

recommenderHybrid5 = ItemKNNScoresHybridTwoRecommender(URM_train, recommenderHybrid4, recommender5)
recommenderHybrid5.fit(alpha = 0.9)

recommenderHybrid6 = ItemKNNScoresHybridTwoRecommender(URM_train, recommenderHybrid5, recommender10)
recommenderHybrid6.fit(alpha = 0.45)

recommenderHybrid7 = ItemKNNScoresHybridTwoRecommender(URM_train, recommenderHybrid6, recommender1)
recommenderHybrid7.fit(alpha = 0.5)

In [32]:
def rank_models(evaluator, recommenders):
    for r in recommenders:
        r_d, _ = evaluator.evaluateRecommender(r)
        print(r.RECOMMENDER_NAME, r_d.loc[10]['MAP'])

In [33]:
rank_models(evaluator_validation, [recommender1, recommender2, recommender3, recommender4, recommender5, recommender6, recommender7, recommender8, recommender9, recommender10])

EvaluatorHoldout: Processed 13649 (100.0%) in 13.69 sec. Users per second: 997
SLIMElasticNetRecommender 0.3433169561814537
EvaluatorHoldout: Processed 13649 (100.0%) in 28.07 sec. Users per second: 486
SLIM_BPR_Recommender 0.3205563114688312
EvaluatorHoldout: Processed 13649 (100.0%) in 14.38 sec. Users per second: 949
RP3betaRecommender 0.3079177903608277
EvaluatorHoldout: Processed 13649 (100.0%) in 19.64 sec. Users per second: 695
ItemKNNCBFRecommender 0.12984797421056535
EvaluatorHoldout: Processed 13649 (100.0%) in 4.18 sec. Users per second: 3264
IALSRecommender 0.3224774011003776
EvaluatorHoldout: Processed 13649 (100.0%) in 21.71 sec. Users per second: 629
P3alphaRecommender 0.2914372179135168
EvaluatorHoldout: Processed 13649 (100.0%) in 10.78 sec. Users per second: 1266
UserKNNCFRecommender 0.3238015431097349
EvaluatorHoldout: Processed 13649 (100.0%) in 3.76 sec. Users per second: 3626
PureSVDRecommender 0.32063983116386297
EvaluatorHoldout: Processed 13649 (100.0%) in 3.61

In [20]:
import numpy as np

# TWO RECOMMENDERS

FROM = 0.45
TO = 0.65
# 2341 at 0.7
def test_percentage(recommender_1, recommender_2, evaluator, step):
    recommender = ItemKNNScoresHybridTwoRecommender(URM_train, recommender_1, recommender_2)
    results = []
    alp_space = np.linspace(FROM, TO, step, True)
    for alp in alp_space:
        recommender.fit(alpha = alp)
        r_d, _ = evaluator.evaluateRecommender(recommender)
        print(alp, ":", r_d.loc[10]['MAP'])
        results.append(r_d.loc[10]['MAP'])
    
    return alp_space, results

In [21]:
STEP = 6
alp_space, results = test_percentage(recommenderHybrid6, recommender1, evaluator_validation, STEP)

EvaluatorHoldout: Processed 3000 (22.0%) in 5.20 min. Users per second: 10
EvaluatorHoldout: Processed 6000 (44.0%) in 10.41 min. Users per second: 10
EvaluatorHoldout: Processed 9000 (65.9%) in 15.72 min. Users per second: 10
EvaluatorHoldout: Processed 12000 (87.9%) in 20.96 min. Users per second: 10
Deallocating Cython objects
EvaluatorHoldout: Processed 13648 (100.0%) in 22.87 min. Users per second: 10
0.45 : 0.24981140907906774
EvaluatorHoldout: Processed 3000 (22.0%) in 5.17 min. Users per second: 10
EvaluatorHoldout: Processed 6000 (44.0%) in 10.60 min. Users per second: 9
EvaluatorHoldout: Processed 9000 (65.9%) in 16.14 min. Users per second: 9
EvaluatorHoldout: Processed 12000 (87.9%) in 21.63 min. Users per second: 9
EvaluatorHoldout: Processed 13648 (100.0%) in 23.69 min. Users per second: 10
0.49 : 0.24986622552987633
EvaluatorHoldout: Processed 3000 (22.0%) in 5.77 min. Users per second: 9
EvaluatorHoldout: Processed 6000 (44.0%) in 11.03 min. Users per second: 9
Evaluato

KeyboardInterrupt: 

In [37]:

evaluator_validation.evaluateRecommender(recommenderHybrid7)

EvaluatorHoldout: Processed 3000 (22.0%) in 5.30 min. Users per second: 9
EvaluatorHoldout: Processed 6000 (44.0%) in 10.54 min. Users per second: 9
EvaluatorHoldout: Processed 9000 (65.9%) in 15.87 min. Users per second: 9
EvaluatorHoldout: Processed 12000 (87.9%) in 21.54 min. Users per second: 9
EvaluatorHoldout: Processed 13649 (100.0%) in 23.56 min. Users per second: 10


(             MAP
 cutoff          
 10      0.344918,
 'CUTOFF: 10 - MAP: 0.3449179, \n')

In [ ]:
import numpy as np

# THREE RECOMMENDERS

FROM = 0.1
TO = 0.9

def test_percentage(recommender_1, recommender_2, recommender_3, evaluator, step):
    recommender = ItemKNNScoresHybridMultipleRecommender(URM_train, recommender_1, recommender_2, recommender_3)
    results = []
    alp_space = np.linspace(12, 18, step, True)
    for alp in alp_space:
        recommender.fit(alpha = alp, beta = 5, gamma = 1)
        r_d, _ = evaluator.evaluateRecommender(recommender)
        print(alp, ":", r_d.loc[10]['MAP'])
        results.append(r_d.loc[10]['MAP'])
    
    return alp_space, results

In [ ]:
alp_space, results = test_percentage(recommender1, recommender2, recommender3, evaluator_validation, 5)

In [ ]:
import matplotlib.pyplot as plt

_ = plt.figure(figsize=(16, 9))
plt.plot(alp_space,results, label='MAP variability')
plt.ylabel('MAP')
plt.xlabel('User Group')
plt.legend()
plt.show()

plt.savefig('userwise.png')

* # SLIMER + KNNweight: 0.2499 with alpha = 0.95
* # SLIMER + SLIMBPR: 0.2507 with alpha = 0.58
* # (SLIMER + SLIMBPR) + KNN_selected_3: 0.2509 with alpha 0.96

In [ ]:
hyp = {'alpha': 0.9674267406603186, 'beta': 0.017675237513072553, 'gamma': 0.04900644363228254}

recommender1 = MultiThreadSLIM_SLIMElasticNetRecommender(URM_all)
# recommender1.fit(**models_to_combine_best['SLIMER'])
# recommender1.save_model(ofp + 'SLIMER_ALL/', 'SLIMER_ALL')
recommender1.load_model(ofp + 'SLIMER_ALL/', 'SLIMER_ALL')

recommender2 = SLIM_BPR_Cython(URM_all)
# recommender2.fit(**models_to_combine_best['SLIMBPR'])
# recommender2.save_model(ofp + 'SLIMBPR_ALL/', 'SLIMBPR_ALL')
recommender2.load_model(ofp + 'SLIMBPR_ALL/', 'SLIMBPR_ALL')

recommender3 = RP3betaRecommender(URM_all)
# recommender3.fit(**models_to_combine_best['RP3beta'])
# recommender3.save_model(ofp + 'RP3beta_ALL/', 'RP3beta_ALL')
recommender3.load_model(ofp + 'RP3beta_ALL/', 'RP3beta_ALL')

recommenderHybrid = ItemKNNScoresHybridMultipleRecommender(URM_all, recommender1, recommender2, recommender3)
recommenderHybrid.fit(**hyp)

recommender = recommenderHybrid

import pandas as pd
at = 10

user_test_path = '../data/data_target_users_test.csv'
user_test_dataframe = pd.read_csv(filepath_or_buffer=user_test_path,
sep=",",
dtype={0:int})

subm_set = user_test_dataframe.to_numpy().T[0]


subm_res = {"user_id":[], "item_list":[]}

for user_id in subm_set:
	subm_res["user_id"].append(user_id)
	res = recommender.recommend(user_id, cutoff=at)
	res = ' '.join(map(str, res))
	if user_id < 3:
		print(user_id)
		print(res)
	subm_res["item_list"].append(res)


	# print(subm_res)

submission = pd.DataFrame.from_dict(subm_res)
	# submission

from datetime import datetime

now = datetime.now() # current date and time


submission.to_csv('../subs/submission {:%Y_%m_%d %H_%M_%S}.csv'.format(now), index=False)



In [ ]:
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
CFrecommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
CFrecommender.load_model('../result_experiments/BEST_SLIM_PURECF/', file_name = CFrecommender.RECOMMENDER_NAME + "_my_own_save.zip")

from Recommenders.FeatureWeighting.CFW_D_Similarity_Linalg import CFW_D_Similarity_Linalg
FW_recommender = CFW_D_Similarity_Linalg(URM_train, ICM_stacked, CFrecommender.W_sparse)
FW_recommender.load_model('../result_experiments/BEST_FW/', file_name = FW_recommender.RECOMMENDER_NAME + "_my_own_save.zip")

In [ ]:
import scipy.sparse as sps
ICM_length_all_5km = ld.getICMlength('5km')
ICM_stacked = sps.hstack((ICM_genre_all, ICM_subgenre_all, ICM_channel_all, ICM_length_all_5km))

In [ ]:
import pandas as pd
argsort_features = np.argsort(-FW_recommender.D_best)

selection_quota = 0.1
        
n_to_select = int(selection_quota*len(argsort_features))
selected_features = argsort_features[:n_to_select]
        
ICM_selected = ICM_stacked.todense()[:,selected_features]
pd.DataFrame(ICM_selected).to_csv('ICM_selected_1.csv', index=False, header=True)


In [ ]:
import scipy.sparse as sps
import pandas as pd
URM_all, ICM_genre_all, ICM_subgenre_all, ICM_channel_all, _ = ld.getCOOs()
ICM_length_all_5km = ld.getICMlength('5km')
pd.DataFrame(sps.hstack([ICM_genre_all, ICM_subgenre_all, ICM_channel_all, ICM_length_all_5km]).todense()).to_csv('ICM_ALL.csv', index=False, header=True)

In [ ]:
recommender1 = recommender1 = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
model_init(recommender1, 'SLIMER', models_to_combine_best['SLIMER'])
rank_models(evaluator_validation, [recommender1])
recommender1.fit(**models_to_combine_best['SLIMER'])
rank_models(evaluator_validation, [recommender1])

# recommender1.save_model(ofp + 'SLIMER_ALL/', 'SLIMER_ALL')
# recommender1.load_model(ofp + 'SLIMER_ALL/', 'SLIMER_ALL')